# Facility Location with Regions

Based on Example 14.3 from the SAS Optimization documentation: https://go.documentation.sas.com/doc/en/pgmsascdc/default/casmopt/casmopt_milpsolver_examples03.htm

For a set of customers and sites, choose which sites to build such that:
- We minimize the sum of the distances between customers and their assigned sites and the building costs of sites
- The capacity for each site is not exceeded
- Sites and customers are in same region


#### Mixed Integer Linear Programming Formulation

$ \begin{array}{llllll} 
\min & \displaystyle \sum _{i \in L} \displaystyle \sum _{j \in F} c_{ij} x_{ij} &+& \displaystyle \sum _{j \in F} f_ j y_ j \\ 
\text{s.t.} & \displaystyle \sum _{j \in F} x_{ij} & = & 1 & \forall i \in L & \text{(assign\_def)} \\ 
& \displaystyle \sum _{i \in L} d_ i x_{ij} & \leq & Cy_ j & \forall j \in F & \text{(capacity)} \\ 
& x_{ij} & = & 0 & \forall i,j \text{ if } r_i \ne r_j & \text{(region\_con)} \\
\\
& x_{ij} \in \{ 0,1\} & & & \forall i \in L, j \in F \\
& y_{j} \in \{ 0,1\} & & & \forall j \in F 
\end{array} 
$


#### Input Data

For the input data we have a comma-separated value (CSV) file with all cities in Germany with more than 50,000 inhabitants retrieved from the German Federal Office of Statistics (www.destatis.de) that also includes geo locations of the cities for plotting. The following code reads the file and plots the data on a map.

In [1]:
import folium
import pandas as pd

# Read the input data and make sure the numbers are all parsed correctly, then print the top of the DataFrame
indata = pd.read_csv('cities_germany.csv', sep=';', decimal=',')
indata["size"] = pd.to_numeric(indata["size"].str.replace(" ", ""), errors='coerce')
indata["density"] = pd.to_numeric(indata["density"].str.replace(" ", ""), errors='coerce')
print(indata)

# Display the cities on a map of Germany
map_input = folium.Map(location=(52, 9), zoom_start=6)
for index, row in indata.iterrows():
    folium.Marker(location=[row["lat"], row["lon"]], tooltip=f'{row["name"]}<br>Size: {row["size"]}<br>Density: {row["density"]}').add_to(map_input)
    
map_input

     state                            name     size  density  zipcode  \
0       11                   Berlin, Stadt  3755251     4214    10178   
1        2   Hamburg, Freie und Hansestadt  1892122     2506    20095   
2        9       München, Landeshauptstadt  1512491     4868    80331   
3        5                     Köln, Stadt  1084831     2678    50667   
4        6        Frankfurt am Main, Stadt   773068     3113    60311   
..     ...                             ...      ...      ...      ...   
190      1                 Elmshorn, Stadt    50772     2377    25335   
191      3                    Emden, Stadt    50535      450    26721   
192      3                   Goslar, Stadt    50203      306    38640   
193      5                  Willich, Stadt    50144      740    47877   
194      8  Heidenheim an der Brenz, Stadt    50025      467    89522   

           lon        lat  type  
0    13.405538  52.517670     1  
1     9.996970  53.550678     1  
2    11.575997  48.13

#### Defining the Optimization Problem with Pyomo

In this specific example, every customer location (city) can also be a site. The demand of each city is its size while we use the density as the cost to build a site. This means that building a site in a less densely populated cities is preferable. This leads to an interesting optimization problem for demonstration purposes but has no real-world meaning.

The code below defines the optimization problem and connects it with the data.

In [2]:
from pyomo.environ import *
from geopy.distance import geodesic

# Set capacity
C = 5000000

# Declare model
model = ConcreteModel()

# Initialize sets
model.SITES = Set(initialize=indata.index.tolist())

# Initialize parameters
model.lat = Param(model.SITES, initialize=indata["lat"])
model.lon = Param(model.SITES, initialize=indata["lon"])
model.demand = Param(model.SITES, initialize=indata["size"], within=Integers)
model.cost = Param(model.SITES, initialize=indata["density"], within=Integers)
model.region = Param(model.SITES, initialize=indata["state"])

# Define sparse set
model.PAIRS = Set(
    initialize=[
        (i, j)
        for i in model.SITES
        for j in model.SITES        
    ]
)

# Define distances
tab = pd.DataFrame(
    [
        [i, j, round(geodesic((model.lat[i], model.lon[i]),(model.lat[j], model.lon[j])).km)]
        for (i, j) in model.PAIRS
    ],
    columns=["i", "j", "dist"],
).set_index(["i", "j"])["dist"]
model.dist = Param(model.PAIRS, initialize=tab)

# Declare variables
model.Assign = Var(model.PAIRS, within=Binary)
model.Build = Var(model.SITES, within=Binary)

# Define constraints
def objective_rule(model):
    return sum(model.dist[i, j] * model.Assign[i, j] for (i, j) in model.PAIRS) + sum(
        model.cost[j] * model.Build[j] for j in model.SITES
    )
model.objective = Objective(rule=objective_rule, sense=minimize)

def assign_def_rule(model, t):
    return sum(model.Assign[i, j] for (i, j) in model.PAIRS if i == t) == 1
model.assign_def = Constraint(model.SITES, rule=assign_def_rule)

def capacity_rule(model, t):
    return (
        sum(model.demand[i] * model.Assign[i, j] for (i, j) in model.PAIRS if j == t)
        <= C * model.Build[t]
    )
model.capacity = Constraint(model.SITES, rule=capacity_rule)

Solve the problem using the SAS solver.

In [3]:
sas_solver = SolverFactory('sas', solver_io="_sascas", hostname='https://my-cas-host.com:443/cas-shared-default-http/', pkce=True)
_ = sas_solver.solve(model, tee=True)

NOTE: Added action set 'optimization'.
NOTE: The problem unknown has 38220 variables (38220 binary, 0 integer, 0 free, 0 fixed).
NOTE: The problem has 390 constraints (195 LE, 195 EQ, 0 GE, 0 range).
NOTE: The problem has 76245 constraint coefficients.
NOTE: The initial MILP heuristics are applied.
NOTE: The MILP presolver value AUTOMATIC is applied.
NOTE: The MILP presolver removed 0 variables and 0 constraints.
NOTE: The MILP presolver removed 0 constraint coefficients.
NOTE: The MILP presolver modified 0 constraint coefficients.
NOTE: The presolved problem has 38220 variables, 390 constraints, and 76245 constraint coefficients.
NOTE: The MILP solver is called.
NOTE: The parallel Branch and Cut algorithm is used.
NOTE: The Branch and Cut algorithm is using up to 16 threads.
         Node   Active   Sols    BestInteger      BestBound      Gap    Time
                0        1      5  63428.0000000              0    63428       0
                0        1      5  63428.0000000   6371

The following code computes the parts of the objective from the output data sets to see how long the total distance between sites and customers is compared to the fixed charge building costs of the sites.

In [4]:
assignments = pd.DataFrame([(indata["lat"][i], indata["lon"][i], indata["lat"][j], indata["lon"][j], model.dist[i,j]) 
                            for (i,j) in model.PAIRS if model.Assign[i,j].value > 0.5],
                            columns=["lat1", "lon1", "lat2", "lon2","distance" ]
                            )
sites = pd.DataFrame([(indata["name"][j], indata["lat"][j], indata["lon"][j], model.cost[j]) 
                      for j in model.SITES if model.Build[j].value > 0.5],
                      columns=["name", "lat", "lon","cost" ]
                    )

# Print the sum of the distances
total_distance = assignments["distance"].sum()
print(f"Total distance: {total_distance}")

## Print the sum of the building costs
total_site_cost = sites["cost"].sum()
print(f"Total site cost: {total_site_cost}")

print(f"Objective: {total_distance + total_site_cost}")

Total distance: 11735
Total site cost: 5197
Objective: 16932


Now display the map with the solution.

In [5]:
map_noregion = folium.Map(location=(52, 9), zoom_start=6)

# Plot all cities
for index, row in indata.iterrows():
    folium.Marker(location=[row["lat"], row["lon"]], tooltip=f'{row["name"]}<br>Size: {row["size"]}<br>Density: {row["density"]}').add_to(map_noregion)

# Plot the cities that are sites to build
for index, row in sites.iterrows():
    folium.Marker(location=[row["lat"], row["lon"]], tooltip=row["name"], icon=folium.Icon(color="green")).add_to(map_noregion)

# Plot the assignments of customers to sites
for idx, row in assignments.iterrows():
    folium.PolyLine([[row["lat1"], row["lon1"]],
                     [row["lat2"], row["lon2"]]]).add_to(map_noregion)

map_noregion

#### Modify the Example to Respect Regions

The following code we redefine the `PAIRS` set and those parts of the model that rely on it to only allow assignments within regions.

Note that by doing this we actually create 16 independent optimization problems, one for each state (some of which are trivial because there is only one choice). The result is an optimization problem with disjoint blocks. By default, the MILP solver detects this structure and deals with it (works better on newer versions). But it's also possible to use this option statement: `options={"with": "milp", "decomp": {"method": "concomp"}}` With these options, the DECOMP algorithm is used that will solve each block independently and in parallel.

The 'with' option chooses the solver to use. Depending on its value, it accepts the same options as the 'solveLp' action or the 'solveMilp' action. The documentation for these can be found here: https://go.documentation.sas.com/doc/en/pgmsascdc/default/casactmopt/casactmopt_optimization_toc.htm

In [6]:
# Declare model
model = ConcreteModel()

# Initialize sets
model.SITES = Set(initialize=indata.index.tolist())

# Initialize parameters
model.lat = Param(model.SITES, initialize=indata["lat"])
model.lon = Param(model.SITES, initialize=indata["lon"])
model.demand = Param(model.SITES, initialize=indata["size"], within=Integers)
model.cost = Param(model.SITES, initialize=indata["density"], within=Integers)
model.region = Param(model.SITES, initialize=indata["state"])

# Define sparse set
model.PAIRS = Set(
    initialize=[
        (i, j)
        for i in model.SITES
        for j in model.SITES
        if model.region[i] == model.region[j]      
    ]
)

# Define distances
tab = pd.DataFrame(
    [
        [i, j, round(geodesic((model.lat[i], model.lon[i]),(model.lat[j], model.lon[j])).km)]
        for (i, j) in model.PAIRS
    ],
    columns=["i", "j", "dist"],
).set_index(["i", "j"])["dist"]
model.dist = Param(model.PAIRS, initialize=tab)

# Declare variables
model.Assign = Var(model.PAIRS, within=Binary)
model.Build = Var(model.SITES, within=Binary)

# Add objective
model.objective = Objective(rule=objective_rule, sense=minimize)

# Add constraints
model.assign_def = Constraint(model.SITES, rule=assign_def_rule)
model.capacity = Constraint(model.SITES, rule=capacity_rule)

# Solve the problem
_ = sas_solver.solve(model, tee=True, options={"with": "milp", "decomp": {"method": "concomp"}})

NOTE: Added action set 'optimization'.
NOTE: The problem unknown has 7766 variables (7766 binary, 0 integer, 0 free, 0 fixed).
NOTE: The problem has 390 constraints (195 LE, 195 EQ, 0 GE, 0 range).
NOTE: The problem has 15337 constraint coefficients.
NOTE: The initial MILP heuristics are applied.
NOTE: The MILP presolver value AUTOMATIC is applied.
NOTE: The MILP presolver removed 8 variables and 8 constraints.
NOTE: The MILP presolver removed 13 constraint coefficients.
NOTE: The MILP presolver modified 116 constraint coefficients.
NOTE: The presolved problem has 7758 variables, 382 constraints, and 15324 constraint coefficients.
NOTE: The MILP solver is called.
NOTE: The Decomposition algorithm is used.
NOTE: The Decomposition algorithm is executing in the distributed computing environment in single-machine mode.
NOTE: The DECOMP method value CONCOMP is applied.
NOTE: The decomposition identification used 0,00 (cpu: 0,00) seconds.
NOTE: The number of block threads has been reduced to

Print the objective for the modified problem.

In [7]:
assignments = pd.DataFrame([(indata["lat"][i], indata["lon"][i], indata["lat"][j], indata["lon"][j], model.dist[i,j]) 
                            for (i,j) in model.PAIRS if model.Assign[i,j].value > 0.5],
                            columns=["lat1", "lon1", "lat2", "lon2","distance" ]
                            )
sites = pd.DataFrame([(indata["name"][j], indata["lat"][j], indata["lon"][j], model.cost[j]) 
                      for j in model.SITES if model.Build[j].value > 0.5],
                      columns=["name", "lat", "lon","cost" ]
                    )

# Print the sum of the distances
total_distance = assignments["distance"].sum()
print(f"Total distance: {total_distance}")

# Print the sum of the building costs
total_site_cost = sites["cost"].sum()
print(f"Total site cost: {total_site_cost}")

print(f"Objective: {total_distance + total_site_cost}")

Total distance: 9970
Total site cost: 18102
Objective: 28072


Display the map for the modified problem where regions are respected. Note that Berlin, Hamburg, and Saarbrücken have to be sites now since they are the only possible sites in their regions. Bremen chooses Bremerhaven as the site in the state of Bremen (which might or might not be a valid decision to make).

In [8]:
map_region = folium.Map(location=(52, 9), zoom_start=6)

# Plot all cities
for index, row in indata.iterrows():
    folium.Marker(location=[row["lat"], row["lon"]], tooltip=f'{row["name"]}<br>Size: {row["size"]}<br>Density: {row["density"]}').add_to(map_region)

# Plot the cities that are sites to build
for index, row in sites.iterrows():
    folium.Marker(location=[row["lat"], row["lon"]], tooltip=row["name"], icon=folium.Icon(color="green")).add_to(map_region)

# Plot the assignments of customers to sites
for idx, row in assignments.iterrows():
    folium.PolyLine([[row["lat1"], row["lon1"]],
                     [row["lat2"], row["lon2"]]]).add_to(map_region)

map_region

In [9]:
# Clean up the SAS connection
del(sas_solver)